In [1]:
import torch
from torch import nn

In [21]:
#tensor of 3 dimensions.
#0th dim: 2 elements (z-dim)
#1st dim: 4 elements (y-dim)
#2nd dim: 5 elements (x-dim, inner most dimension)


## PyTorch modelling
* torch.nn - contains all buildings for computational graphs (neural network is a type of graph)
* torch.nn.Parameter - what parameters should our model try and learn, often a PyTorch layer from torch.nn will set these for us. 
* torch.nn.Module - The base class for all neural network modules. If it is subclass-ed, please overwrite the "forward()" funtion
* torch.optim - optimizers. Helps with gradient descent. 
*requires_grad - When calling tensor.backward(), pytorch travels the graph in the reverse direction to help compute a new gradient.

In [ ]:
class LinearRegressionMod(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1, requires_grad=True)) #parameter
        self.bias = nn.Parameter(torch.randn1, requires_grad=True) #parameter.

        #usuall in a NN, PyTorch later from torch.nn will set these for us. 

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias